In [9]:
# Standard Python libraries
import os  # For operating system dependent functionality
import random  # For generating random numbers

# Data handling and manipulation libraries
import numpy as np  # For numerical operations on arrays

# PIL - Python Imaging Library
from PIL import Image  # For image processing

# PyTorch - Deep learning framework
import torch  # Main PyTorch library
import torch.nn as nn  # For building neural network layers
import torch.optim as optim  # For optimization algorithms
from torch.utils.data import DataLoader, Dataset, random_split  # Utilities for data handling in PyTorch

# torchvision - Utilities for working with image data and pretrained models
from torchvision import transforms  # For image transformations
from torchvision.models import resnet152, ResNet152_Weights  # Pretrained models and their weights

# Scikit-learn - Machine learning library
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix  # Metrics for evaluating model performance

# tqdm - Library for progress bars
from tqdm import tqdm  # For displaying progress bars in loops

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, classification_report
from sklearn.preprocessing import label_binarize
from itertools import cycle
from scipy import interp



In [10]:
def setup_device():
    """Set up the PyTorch device - use GPU if available, otherwise CPU."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Current device is : {device}")
    return device

# Call the function to set up the device
device = setup_device()



Current device is : cuda


In [11]:
transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [12]:
class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        # Initialize dataset with data, labels, and optional transforms
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        # Return number of items in dataset
        return len(self.labels)

    def __getitem__(self, idx):
        # Get image and label by index

        # Reshape image data and convert to PIL Image
        image = self.data[:, idx].reshape(300, 300, 3)
        image = Image.fromarray(image)

        # Apply transform if provided
        if self.transform:
            image = self.transform(image)

        # Get label for the image
        label = self.labels[idx]

        return image, label

## Loading the Test set(change the file path for test set accordingly)

In [13]:
def load_data(file_path):
    """ Load data from a .npy file. """
    return np.load(file_path)

def create_dataset(data, labels, transform=None):
    """ Create a custom dataset using the loaded data and labels along with the predefined transformations. """
    return CustomDataset(data, labels, transform=transform)

def create_data_loader(dataset, batch_size):
    """ Create a DataLoader for the given dataset with the specified batch size. """
    return DataLoader(dataset, batch_size=batch_size)

# Define file paths for the dataset and corresponding labels
test_data_filepath = 'data-1.npy'# File path to data of test set
test_labels_filepath = 'labels-1.npy' # File path to labels of test set

# Load data and labels
data = load_data(test_data_filepath) # Shape of data is Flattend Image data X Number of Samples
labels = load_data(test_labels_filepath)


# (Assuming 'transform' is predefined somewhere in your code)
# Create a custom dataset
dataset = create_dataset(data, labels, transform=transform)

# Define batch size
BATCH_SIZE = 128

# Create a DataLoader
test_loader = create_data_loader(dataset, BATCH_SIZE)

## Testing the Model(change the file path for saved model accordingly)

In [17]:
def load_model(model_path, num_classes):
    # Load a ResNet152 model with a specified number of classes.
    model = resnet152(pretrained=False, num_classes=num_classes)
    model.load_state_dict(torch.load(model_path))
    return model

def evaluate_model(model, test_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    y_pred = []
    y_true = []
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device).float()  # Ensure data is in float32
            target = target.to(device).long()  # Ensure target labels are in long

            output = model(data)

            # Convert model output to float32 if not already
            output = output.float()

            test_loss += criterion(output, target).item()

            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

            y_pred.extend(predicted.view(-1).cpu().numpy())
            y_true.extend(target.view(-1).cpu().numpy())

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100 * correct / total

    # Generate classification report
    report = classification_report(y_true, y_pred, target_names=[class_mapping[i] for i in range(NUM_CLASSES)])

    return test_loss, test_accuracy, report



# Define paths, constants, and class mapping.
MODEL_PATH = 'model saves/Best Model 6/model_epoch_10.pth'
NUM_CLASSES = 9
class_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', -1: 'Unknown'}

# Load model and define loss criterion.
model = load_model(MODEL_PATH, NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()

# Evaluate the model.
test_loss, test_accuracy , report = evaluate_model(model, test_loader, criterion, device)

# Print the test loss and accuracy.
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')
# Print the test loss, accuracy, and classification report
print("Classification Report:")
print(report)


/apps/pytorch/2.0.1/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/apps/pytorch/2.0.1/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Test Loss: 0.0002, Test Accuracy: 99.26%
Classification Report:
              precision    recall  f1-score   support

           A       1.00      1.00      1.00        30
           B       1.00      1.00      1.00        30
           C       1.00      0.97      0.98        30
           D       1.00      1.00      1.00        30
           E       1.00      1.00      1.00        30
           F       1.00      1.00      1.00        30
           G       0.94      1.00      0.97        30
           H       1.00      0.97      0.98        30
           I       1.00      1.00      1.00        30

    accuracy                           0.99       270
   macro avg       0.99      0.99      0.99       270
weighted avg       0.99      0.99      0.99       270

